In [1]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.utils import get_config, read_file

In [4]:
config = get_config.read_yaml()
symbols = config['data']['symbols']
symbols

['ETH/USDT',
 'BTC/USDT',
 'BNB/USDT',
 'SOL/USDT',
 'ADA/USDT',
 'LINK/USDT',
 'XRP/USDT',
 'LTC/USDT',
 'UNI/USDT']

In [12]:
data = read_file.read_merged_training_data()
data

Reading merged Training data: merged_training_data !
Successfully read: merged_training_data !


,"('open', 'ETH')","('high', 'ETH')","('low', 'ETH')","('close', 'ETH')","('volume', 'ETH')","('rsi', 'ETH')","('sma-50', 'ETH')","('sma-100', 'ETH')","('sma-200', 'ETH')","('ema-50', 'ETH')",...,"('volume', 'UNI')","('rsi', 'UNI')","('sma-50', 'UNI')","('sma-100', 'UNI')","('sma-200', 'UNI')","('ema-50', 'UNI')","('ema-100', 'UNI')","('ema-200', 'UNI')","('atr', 'UNI')","('adx', 'UNI')"
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-09-17 00:00:00,0.030328,0.034286,0.030840,0.032922,0.222320,0.596522,0.031564,0.034522,0.038023,0.032233,...,0.121986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-17 04:00:00,0.032922,0.033792,0.033686,0.033543,0.116355,0.619531,0.031755,0.034397,0.037953,0.032325,...,0.503877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-17 08:00:00,0.033546,0.034118,0.033324,0.034231,0.122179,0.644432,0.031975,0.034273,0.037887,0.032442,...,0.374410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-17 12:00:00,0.034231,0.034021,0.033592,0.033612,0.135888,0.610914,0.032145,0.034149,0.037819,0.032529,...,0.361967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-17 16:00:00,0.033613,0.035363,0.034185,0.035557,0.123050,0.679286,0.032332,0.034024,0.037765,0.032692,...,0.463514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-29 04:00:00,0.683935,0.685560,0.693684,0.686923,0.012268,0.491064,0.717952,0.748766,0.805454,0.726841,...,0.005838,0.317463,0.184519,0.197453,0.210344,0.187205,0.196637,0.197513,0.053775,0.228563
2024-06-29 08:00:00,0.686890,0.685071,0.694923,0.687081,0.009915,0.492804,0.717500,0.748469,0.804836,0.726563,...,0.005479,0.280237,0.183995,0.197222,0.210036,0.186608,0.196201,0.197280,0.051363,0.236403
2024-06-29 12:00:00,0.687048,0.684457,0.694000,0.685820,0.008350,0.479484,0.717140,0.748087,0.804218,0.726243,...,0.005493,0.311841,0.183564,0.196938,0.209727,0.186080,0.195798,0.197063,0.049421,0.248864


In [13]:
def generate_position(data):
    for symbol in symbols:
        symbol = symbol.split('/')[0]
        col = ('position', symbol)
        data[col] = 0.11
    return data

In [15]:
data = generate_position(data)
data

,"('open', 'ETH')","('high', 'ETH')","('low', 'ETH')","('close', 'ETH')","('volume', 'ETH')","('rsi', 'ETH')","('sma-50', 'ETH')","('sma-100', 'ETH')","('sma-200', 'ETH')","('ema-50', 'ETH')",...,"('adx', 'UNI')","(position, ETH)","(position, BTC)","(position, BNB)","(position, SOL)","(position, ADA)","(position, LINK)","(position, XRP)","(position, LTC)","(position, UNI)"
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-09-17 00:00:00,0.030328,0.034286,0.030840,0.032922,0.222320,0.596522,0.031564,0.034522,0.038023,0.032233,...,NaN,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
2020-09-17 04:00:00,0.032922,0.033792,0.033686,0.033543,0.116355,0.619531,0.031755,0.034397,0.037953,0.032325,...,NaN,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
2020-09-17 08:00:00,0.033546,0.034118,0.033324,0.034231,0.122179,0.644432,0.031975,0.034273,0.037887,0.032442,...,NaN,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
2020-09-17 12:00:00,0.034231,0.034021,0.033592,0.033612,0.135888,0.610914,0.032145,0.034149,0.037819,0.032529,...,NaN,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
2020-09-17 16:00:00,0.033613,0.035363,0.034185,0.035557,0.123050,0.679286,0.032332,0.034024,0.037765,0.032692,...,NaN,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-29 04:00:00,0.683935,0.685560,0.693684,0.686923,0.012268,0.491064,0.717952,0.748766,0.805454,0.726841,...,0.228563,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
2024-06-29 08:00:00,0.686890,0.685071,0.694923,0.687081,0.009915,0.492804,0.717500,0.748469,0.804836,0.726563,...,0.236403,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
2024-06-29 12:00:00,0.687048,0.684457,0.694000,0.685820,0.008350,0.479484,0.717140,0.748087,0.804218,0.726243,...,0.248864,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
